In [2]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver

In [123]:
inn_list_1 = ['7703761570', '7710746170', '7702775114', '3015099975']  
#добавьте свои ИНН в списочек
#если значения строковые, можно поступить вот так:
# inn_list = [7703761570,7710746170,7702775114 ]
# inn_list = [str(i) for i in input().split(' ')]
# inn_list_1 = []
# for i in inn_list:
#     if len(i)<2:
#         pass
#     else:
#         inn_list_1.append(i)

#выйти из режима комментария можно cntrl+A --> cntrl+L

Чтобы все работало, geckodriver можно загрузить по этой ссылке: https://github.com/mozilla/geckodriver/releases

In [ ]:
browser = webdriver.Firefox(executable_path=r'C:\Users\Boss\Desktop\geckodriver-v0.25.0-win64\geckodriver.exe')
#здесь укажите СВОЙ путь для файла, куда вы распаковали приложение
#не забудьте про само имя geckodriver.exe

In [ ]:
url = 'http://www.spark-interfax.ru/ru/integration'

In [49]:
browser.get(url) #здесь магия, когда браузер запустится сам и перейдет на ссылочку

In [ ]:
#возможно, будет хорошо подклюить рандом для времени
import time
import random 
k = random.randint(3,15) #чтобы браузер засыпал каждый раз на разное количество секунд из списка

In [119]:
#вводим нужный нам ИНН

url = 'http://www.spark-interfax.ru/ru/integration'
links = [] #создадим глобальный список, куда 

def extract_links(inn_list, links=links):
    for inn in inn_list_1:

        browser.get(url)
        #жмакаем на кнопочку поиска
        button_enter = browser.find_element_by_css_selector('div.col-sm-8:nth-child(1)').click() 
        browser.find_element_by_css_selector('.search-form__input').clear() #на случай,если мы что-то написали
        browser.find_element_by_css_selector('.search-form__input').send_keys(inn)
        time.sleep(k)  #можно поставить рандмайзеры, чтобы браузер меньше
        for i in browser.find_elements_by_class_name('autocomplete-suggestion'):
            links.append(i.find_element_by_css_selector('a').get_attribute('href'))
            time.sleep(k)
    return links


In [120]:
extract_links(inn_list_1)

['http://www.spark-interfax.ru/moskva-presnenski/ooo-donats-kafe-inn-7703761570-ogrn-1127746039486-27ec078738f345d9bd3606832593025e',
 'http://www.spark-interfax.ru/moskva-tverskoi/ooo-burger-king-rus-inn-7710746170-ogrn-1097746064074-6178897a9d687e1ce0531b9aa8c027ab',
 'http://www.spark-interfax.ru/moskva-basmanny/ooo-muzykalny-leibl-blek-star-inn-7702775114-ogrn-1117746914350-714124d0516f4b0cb2c7a0200c7d959b',
 'http://www.spark-interfax.ru/astrakhanskaya-oblast-astrakhan/ooo-spisok-inn-3015099975-ogrn-1133015001790-fcf39cc6d4dd4b068184e0534d719575']

In [121]:
links

['http://www.spark-interfax.ru/moskva-presnenski/ooo-donats-kafe-inn-7703761570-ogrn-1127746039486-27ec078738f345d9bd3606832593025e',
 'http://www.spark-interfax.ru/moskva-tverskoi/ooo-burger-king-rus-inn-7710746170-ogrn-1097746064074-6178897a9d687e1ce0531b9aa8c027ab',
 'http://www.spark-interfax.ru/moskva-basmanny/ooo-muzykalny-leibl-blek-star-inn-7702775114-ogrn-1117746914350-714124d0516f4b0cb2c7a0200c7d959b',
 'http://www.spark-interfax.ru/astrakhanskaya-oblast-astrakhan/ooo-spisok-inn-3015099975-ogrn-1133015001790-fcf39cc6d4dd4b068184e0534d719575']

Видим, что тела ссылок уже достаточно для нужной информации, оформими красиво:

In [129]:
ogrn_dict = {}
for i in links:
    number_inn = i.find('inn')
    inn = i[number_inn+5:number_inn+14]
    number_ogrn = i.find('ogrn')
    ogrn = i[number_ogrn+5:number_ogrn+18]
    ogrn_dict[inn] = ogrn
    
   

In [130]:
ogrn_dict #готово

{'703761570': '1127746039486',
 '710746170': '1097746064074',
 '702775114': '1117746914350',
 '015099975': '1133015001790'}

для извлечения дополнительной инфы можно пойти дальше:

In [131]:
inn_base = {}
def extract_info(links,inn_base = inn_base ):
    
    for link in links:
        browser.get(link)
        time.sleep(k) #помним про рандомизированное время
        for i in browser.find_elements_by_class_name('company-description__text'):
            print(i.text)
            a = i.text
            
            number_inn = link.find('inn')
            inn = link[number_inn+5:number_inn+14]
            
            inn_base[inn] = a
            time.sleep(k)
    return inn_base

In [133]:
extract_info(links)

Компания ООО ДОНАТС КАФЕ, адрес: г. Москва, ул. Красина, д. 21 стр. 1 зарегистрирована 27.01.2012. Организации присвоены ИНН 7703761570, ОГРН 1127746039486, КПП 770301001. Основным видом деятельности является деятельность ресторанов и кафе с полным ресторанным обслуживанием, кафетериев, ресторанов быстрого питания и самообслуживания, всего зарегистрировано 8 видов деятельности по ОКВЭД. Связи с другими компаниями отсутствуют.
Количество совладельцев (по данным ЕГРЮЛ): 3, генеральный директор - Макаров Дмитрий Владимирович. Размер уставного капитала 1 000 000₽.
Компания ООО ДОНАТС КАФЕ не принимала участие в тендерах. В отношении компании было возбуждено 12 исполнительных производств. ООО ДОНАТС КАФЕ участвовало в 9 арбитражных делах: в 1 в качестве истца, и в 8 в качестве ответчика.
Реквизиты ООО ДОНАТС КАФЕ, юридический адрес, официальный сайт и выписка ЕГРЮЛ доступны в системе СПАРК (демо-доступ бесплатно).
Компания ООО БУРГЕР КИНГ (РУС), адрес: г. Москва, пер. Лесной 4-Й, д. 4 офис 

{'703761570': 'Компания ООО ДОНАТС КАФЕ, адрес: г. Москва, ул. Красина, д. 21 стр. 1 зарегистрирована 27.01.2012. Организации присвоены ИНН 7703761570, ОГРН 1127746039486, КПП 770301001. Основным видом деятельности является деятельность ресторанов и кафе с полным ресторанным обслуживанием, кафетериев, ресторанов быстрого питания и самообслуживания, всего зарегистрировано 8 видов деятельности по ОКВЭД. Связи с другими компаниями отсутствуют.\nКоличество совладельцев (по данным ЕГРЮЛ): 3, генеральный директор - Макаров Дмитрий Владимирович. Размер уставного капитала 1 000 000₽.\nКомпания ООО ДОНАТС КАФЕ не принимала участие в тендерах. В отношении компании было возбуждено 12 исполнительных производств. ООО ДОНАТС КАФЕ участвовало в 9 арбитражных делах: в 1 в качестве истца, и в 8 в качестве ответчика.\nРеквизиты ООО ДОНАТС КАФЕ, юридический адрес, официальный сайт и выписка ЕГРЮЛ доступны в системе СПАРК (демо-доступ бесплатно).',
 '710746170': 'Компания ООО БУРГЕР КИНГ (РУС), адрес: г. 

вжух!

In [134]:
inn_base

{'703761570': 'Компания ООО ДОНАТС КАФЕ, адрес: г. Москва, ул. Красина, д. 21 стр. 1 зарегистрирована 27.01.2012. Организации присвоены ИНН 7703761570, ОГРН 1127746039486, КПП 770301001. Основным видом деятельности является деятельность ресторанов и кафе с полным ресторанным обслуживанием, кафетериев, ресторанов быстрого питания и самообслуживания, всего зарегистрировано 8 видов деятельности по ОКВЭД. Связи с другими компаниями отсутствуют.\nКоличество совладельцев (по данным ЕГРЮЛ): 3, генеральный директор - Макаров Дмитрий Владимирович. Размер уставного капитала 1 000 000₽.\nКомпания ООО ДОНАТС КАФЕ не принимала участие в тендерах. В отношении компании было возбуждено 12 исполнительных производств. ООО ДОНАТС КАФЕ участвовало в 9 арбитражных делах: в 1 в качестве истца, и в 8 в качестве ответчика.\nРеквизиты ООО ДОНАТС КАФЕ, юридический адрес, официальный сайт и выписка ЕГРЮЛ доступны в системе СПАРК (демо-доступ бесплатно).',
 '710746170': 'Компания ООО БУРГЕР КИНГ (РУС), адрес: г. 